In [ ]:
import numpy as np
import scanpy as sc
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sys
path_helper = ["C:\\","Users","vfriedrich","projects","monkey_IZI","git_documentation","scRNAseq_cross_species_primate_human","analysis","helper"]
sys.path.append(os.path.join(*path_helper))
import helperVDF as h
print(sys.executable)
from matplotlib.colors import Normalize

In [ ]:
pre = "MH75"
drive = 'F'
species = 'cyno'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                                            pre = pre,
                                                                                            add_path = True)

In [ ]:
_,base_table_path_M71,_,_ = h.return_local_paths(drive = drive,pre = "M71",add_path = False)
df_dge_M = pd.read_csv(os.path.join(base_table_path_M71,'M71_dge_tab_edgeR.csv'),index_col=0)

_,base_table_path_H71,_,_ = h.return_local_paths(drive = drive,pre = "H71",add_path = False)
df_dge_H = pd.read_csv(os.path.join(base_table_path_H71,'H71_dge_tab_edgeR.csv'),index_col=0)

df_dge_H['FDR_0.25'] = df_dge_H['FDR'] <= 0.25
df_dge_H['FDR_0.5'] = df_dge_H['FDR'] <= 0.5
df_dge_H['FDR_0.2'] = df_dge_H['FDR'] <= 0.2

df_dge_M['FDR_0.25'] = df_dge_M['FDR'] <= 0.25
df_dge_M['FDR_0.5'] = df_dge_M['FDR'] <= 0.5
df_dge_M['FDR_0.2'] = df_dge_M['FDR'] <= 0.2

In [ ]:
list(set(pd.unique(df_dge_M['celltype'])) &set(pd.unique(df_dge_H['celltype'])))

In [ ]:
warnings.filterwarnings('ignore')
columns_df = ['celltype','comparison_vs_00hr_H','comparison_vs_00hr_M','nr_FDR_cyno','nr_FDR_human','nr_FDR_both','fraction_same_direction',
              'mean_diff_logFCM','median_diff_logFCM']
df_DGE_processed = pd.DataFrame(columns=columns_df)
for celltype in list(set(pd.unique(df_dge_M['celltype'])) &set(pd.unique(df_dge_H['celltype']))):
    for comparison_vs_00hr_H in ['timepoints06hr','timepoints24hr']:
        for comparison_vs_00hr_M in ['timepoints06hr','timepoints24hr']:
            df_dge_H_ct = df_dge_H[(df_dge_H['comparison.vs.00hr'] == comparison_vs_00hr_H)&(df_dge_H['celltype'] == celltype)&(df_dge_H['species'] == 'human')]
            df_dge_H_ct_FDR = df_dge_H_ct[df_dge_H_ct['FDR_0.5'] == True]

            df_dge_M_ct = df_dge_M[(df_dge_M['comparison.vs.00hr'] == comparison_vs_00hr_M)&(df_dge_M['celltype'] == celltype)&(df_dge_M['species'] == 'cyno')]
            df_dge_M_ct_FDR = df_dge_M_ct[df_dge_M_ct['FDR_0.5'] == True]

            cyno_genes = list(df_dge_M_ct_FDR['genes']) 

            human_genes = list(df_dge_H_ct_FDR['genes']) 

            joined_genes = list(set(cyno_genes).intersection(human_genes))


            df_dge_M_ct_FDR_joined_genes = df_dge_M_ct_FDR[df_dge_M_ct_FDR['genes'].isin(joined_genes)]

            df_dge_H_ct_FDR_joined_genes = df_dge_H_ct_FDR[df_dge_H_ct_FDR['genes'].isin(joined_genes)]

            df_dge_H_ct_FDR_joined_genes.rename(columns={'logFC': 'logFC_H'}, inplace=True)
            df_dge_M_ct_FDR_joined_genes.rename(columns={'logFC': 'logFC_M'}, inplace=True)

            merged_df = pd.merge(df_dge_M_ct_FDR_joined_genes, df_dge_H_ct_FDR_joined_genes, on=['genes'])

            #fraction_same_direction = sum(merged_df['logFC_M']*merged_df['logFC_H']>0)/len(merged_df)

            fraction_same_direction = np.mean(np.sign(merged_df['logFC_M']) ==  np.sign(merged_df['logFC_H']))
            mean_diff_logFCM = np.mean(np.abs(merged_df['logFC_H'] - merged_df['logFC_M']))
            median_diff_logFCM = np.median(np.abs(merged_df['logFC_H'] - merged_df['logFC_M']))
            df_DGE_processed= df_DGE_processed.append(pd.DataFrame([celltype,
                                   comparison_vs_00hr_H,comparison_vs_00hr_M,
                                   len(cyno_genes),
                                   len(human_genes),
                                   len(joined_genes),
                                   fraction_same_direction,
                                mean_diff_logFCM,median_diff_logFCM],index=columns_df).T)

df_DGE_processed['comp'] = df_DGE_processed['comparison_vs_00hr_H'].str[-4:] + '_' + df_DGE_processed['comparison_vs_00hr_M'].str[-4:]

df_DGE_processed['ct+comp_00hr_H'] = df_DGE_processed['celltype'].astype(str) + '_' + df_DGE_processed['comparison_vs_00hr_H'].str[-4:]

df_DGE_processed['ct+comp_00hr_M'] = df_DGE_processed['celltype'].astype(str) + '_' + df_DGE_processed['comparison_vs_00hr_M'].str[-4:]

df_DGE_processed['comp_00hr_M'] = df_DGE_processed['comparison_vs_00hr_M'].str[-4:]

df_DGE_processed['negative_mean_diff_logFCM'] =-df_DGE_processed['mean_diff_logFCM']            
df_DGE_processed.to_csv(os.path.join(base_table_path,pre + '_df_DGE_processed.csv'))

In [ ]:
df_DGE_processed

In [ ]:
plt.figure(figsize=(14, 4))
# Set the aesthetic style of the plots
sns.set(style="whitegrid")

# Create a scatter plot with seaborn for categorical H and M
scatter = sns.scatterplot(
    x='ct+comp_00hr_H',
    y='comp_00hr_M',
    size='negative_mean_diff_logFCM',
    hue='fraction_same_direction',
    data=df_DGE_processed,
    palette='Purples',
    sizes=(100, 800),
    alpha=1,
    edgecolor='black',
    legend=True
)

ax = plt.gca()

# Add color bar for 'fraction_same_direction'
#norm = plt.Normalize(0,1)
sm = plt.cm.ScalarMappable(cmap="Purples")
#sm.set_array([])
#cbar = plt.colorbar(sm, ax=ax)
#cbar.set_label('Fraction Same Direction')
#plt.colorbar(sm, label='Fraction Same Direction')
ax.set_ylim(-0.5, 1.5)
# Add labels and title
plt.xlabel('Human')
plt.ylabel('Cyno')
plt.title('DGE summary')
ax.legend(bbox_to_anchor=(1.05, 1))
plt.subplots_adjust(right=0.75)
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig(os.path.join(base_plots_path,pre + '_scatter_same_direction_logFCM.pdf'),bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(14, 4))
# Set the aesthetic style of the plots
sns.set(style="whitegrid")

# Create a scatter plot with seaborn for categorical H and M
scatter = sns.scatterplot(
    x='ct+comp_00hr_H',
    y='comp_00hr_M',
    size='nr_FDR_both',
    hue='fraction_same_direction',
    data=df_DGE_processed,
    palette='Purples',
    sizes=(100, 800),
    alpha=1,
    edgecolor='black',
    legend=True
)

ax = plt.gca()

# Add color bar for 'fraction_same_direction'
#norm = plt.Normalize(0,1)
sm = plt.cm.ScalarMappable(cmap="Purples")
#sm.set_array([])
#cbar = plt.colorbar(sm, ax=ax)
#cbar.set_label('Fraction Same Direction')
#plt.colorbar(sm, label='Fraction Same Direction')
ax.set_ylim(-0.5, 1.5)
# Add labels and title
plt.xlabel('Human')
plt.ylabel('Cyno')
plt.title('DGE summary')
ax.legend(bbox_to_anchor=(1.05, 1))
plt.xticks(rotation=90)
plt.subplots_adjust(right=0.75)
plt.tight_layout()
plt.savefig(os.path.join(base_plots_path,pre + '_scatter_same_direction_nr_FDR_both.pdf'),bbox_inches='tight')
plt.show()

### Save session

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)
h.print_main_versions()